# Input setup

In [17]:
import torch
import cv2
import numpy as np

cap = cv2.VideoCapture('./testVideos/qutusubu.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))


# YOLO

In [18]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /Users/vatsal007/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-3-10 Python-3.8.16 torch-2.1.0.dev20230302 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


# Sort

In [19]:
from sort.sort import Sort

# Set up the SORT tracker
tracker = Sort(max_age=10000)

# Detection and Display

In [20]:
import time
start_time = time.time()
num_frames = 0

ids = []
while(cap.isOpened()):
      
# Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        # Increment the frame count
        num_frames += 1
        # Calculate fps
        elapsed_time = time.time() - start_time
        fps = num_frames / elapsed_time
        # Display the fps
        cv2.putText(frame, f"FPS: {round(fps, 2)}", (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        #detecting any object
        img = []
        img.append(frame)
        result = model(img).pandas().xyxy[0]

        #converting output to sort input
        boxes = []
        for i in result.index:
            if(result['name'][i] == 'car' or result['name'][i] == 'truck' or result['name'][i] == 'motorcycle' or result['name'][i] == 'bicycle'):
                xleft,yleft,xright,yright = int(result['xmin'][i]),int(result['ymin'][i]),int(result['xmax'][i]),int(result['ymax'][i])
                bbox = [xleft, yleft, xright, yright]
                boxes.append(bbox)
        boxes = np.array(boxes)
        # features = np.array(encoder(frame,boxes))
        
        # Use SORT to track the cars across frames
        track_bbs_ids = tracker.update(boxes)
        for track in track_bbs_ids:
            xmin, ymin, xmax, ymax, track_id = track
            ids.append(track_id)
            cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255,0,0), 2)
            cv2.putText(frame, str(track_id), (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)


        cv2.putText(frame, "Total Cars: "+str(len(set(ids))), (10,10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
        cv2.imshow('Frame', frame)
            

    # Press Q on keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
  
# Break the loop
    else:
        break

1   HIToolbox                           0x0000000191fded4c _ZN15MenuBarInstance21IsAutoShowHideAllowedEv + 284
2   HIToolbox                           0x0000000191eb8230 _ZN15MenuBarInstance24UpdateAutoShowVisibilityE5Pointh + 40
3   HIToolbox                           0x0000000191e2530c _ZN15MenuBarInstance16ForEachMenuBarDoEU13block_pointerFvPS_E + 72
4   HIToolbox                           0x0000000191eb88a0 _ZN15MenuBarInstance20AutoShowHideObserverEjP14OpaqueEventRefPv + 216
5   HIToolbox                           0x0000000191e1bcd0 _NotifyEventLoopObservers + 176
6   HIToolbox                           0x0000000191e4fbb0 PostEventToQueueInternal + 696
7   HIToolbox                           0x0000000191e51340 _ZL29CreateAndPostEventWithCGEventP9__CGEventjhP17__CFMachPortBoost + 460
8   HIToolbox                           0x0000000191e50c44 _ZL38PullEventsFromWindowServerOnConnectionjhP17__CFMachPortBoost + 296
9   HIToolbox                           0x0000000191e50ae0 _ZL14Messag

: 